In [1]:
from pvlib.location import Location
from dotenv import load_dotenv
import googlemaps
from googlemaps.client import Client

import pandas as pd
import os





In [17]:
# google map API adaptor
from pydantic import BaseModel
from googlemaps.elevation import elevation
from googlemaps.timezone import timezone
from googlemaps.exceptions import ApiError, HTTPError, Timeout, TransportError
from datetime import datetime
from zoneinfo import ZoneInfo

class GoogleTimeZone(BaseModel):
    dstOffset: int | None
    rawOffset: int | None
    status: str
    timeZoneId: str | None
    timeZoneName: str | None
    def __init__(self, **kwarg):
        super().__init__(**kwarg)


class GoogleMap_Adaptor:
    def __init__(self, client: Client) -> None:
        self.__client = client
    
    def get_timezone(self, latitude: float, longitude: float) -> GoogleTimeZone|None:
        try:
            result = timezone(client=self.__client, location=(latitude, longitude),timestamp=datetime.now(tz=ZoneInfo('UTC')))
            return GoogleTimeZone(**result)
        except (ApiError, HTTPError, Timeout, TransportError) as e:
            print(f"Error fetching timezone: {str(e)}")
            return None
    
    def get_altitude(self, latitude: float, longitude: float) -> float | None:
        try:
            result = elevation(client=self.__client, locations=(latitude, longitude))
            return result[0]['elevation']
        except (ApiError, HTTPError, Timeout, TransportError) as e:
            print(f"Error fetching altitude: {str(e)}")
            return None
        except (IndexError, KeyError) as e:
            print(f"Error processing altitude data: {str(e)}")
            return None

In [18]:
# open weather API adaptor
from pydantic import BaseModel
from enum import Enum
import requests
class OpenWeather_General(BaseModel):
    temp:float|None
    feels_like:float|None
    temp_min:float|None
    temp_max:float|None
    pressure:float|None
    humidity:float |None
    sea_level:float|None
    grnd_level:float|None
    
    def __init__(self,**kwarg):
        super().__init__(**kwarg)

class OpenWeather_Wind(BaseModel):
    speed:float|None
    deg:float|None
    gust:float|None
    def __init__(self,**kwarg):
        super().__init__(**kwarg)

class OpenWeather_Unit(Enum):
    STANDARD='standard'
    METRIC='metric'
    IMPERIAL='imperial'

class OpenWeather_Adaptor:

    def __init__(self,apikey:str,unit:OpenWeather_Unit):
        self.__apikey=apikey
        self.unit=unit
    
    def get_currentWeather(self, latitude: float, longitude: float) -> tuple[OpenWeather_General, OpenWeather_Wind]:
        try:
            res = requests.get("https://api.openweathermap.org/data/2.5/weather", params={
                "lat": latitude,
                "lon": longitude,
                "appid": self.__apikey,
                "units": self.unit.value
            })
            res.raise_for_status()  # Raise an exception for bad status codes
            result = res.json()
            general_data = OpenWeather_General(**result['main'])
            wind_data = OpenWeather_Wind(**result["wind"])
            return general_data, wind_data
        except requests.RequestException as e:
            print(f"Error fetching weather data: {e}")
            return None, None
        except KeyError as e:
            print(f"Error parsing weather data: {e}")
            return None, None
        except Exception as e:
            print(f"Unexpected error: {e}")
            return None, None


In [19]:
# Solcast API adaptor

class Solcast_Adaptor:
    def __init__(self,apikey:str):
        self.__apikey=apikey

    def get_estimated_actuals(self,latitude:float,longitude:float)->pd.DataFrame|None:
        try:
            response=requests.get(f"https://api.solcast.com.au/world_radiation/estimated_actuals",
                                  params={
                                      "api_key":self.__apikey,
                                      "latitude": latitude,
                                      "longitude":longitude,
                                      "hours":1,
                                      "format":'json'
                                  })
            response.raise_for_status()
            result=response.json()
            return pd.DataFrame(data=result["estimated_actuals"])
        except requests.RequestException as e:
            print(f"Error fetching the irradiance data: {e}")
            return None
        except KeyError as e:
            print(f"Error parsing the irradiance data: {e}")
            return None
        except Exception as e:
            print(f"unexpected exception in `get_estimated_actual`:{e}")
            return None
        



In [20]:
# initialize google client
load_dotenv()
google_apikey=os.getenv("google_api_key")
openWeather_apikey=os.getenv("open_weather_api")
solcast_apikey=os.getenv('solcast_apikey')
googlemap_client=googlemaps.Client(key=google_apikey)
google_adaptor=GoogleMap_Adaptor(client=googlemap_client)
openweather_adaptor=OpenWeather_Adaptor(apikey=openWeather_apikey,unit=OpenWeather_Unit.METRIC)
solcast_adaptor=Solcast_Adaptor(apikey=solcast_apikey)

In [21]:
# retrieve and process JSON data
# Location is in Decimal Degree coordinates
coordinates1={"lat":54.3395,"lon":-114.8016}
altitude=google_adaptor.get_altitude(coordinates1["lat"],coordinates1['lon'])
tz=google_adaptor.get_timezone(coordinates1["lat"],coordinates1["lon"])
location1=Location(latitude=coordinates1["lat"],longitude=coordinates1["lon"],tz=tz.timeZoneId,altitude=altitude)
location1



Location: 
  name: None
  latitude: 54.3395
  longitude: -114.8016
  altitude: 635.3804321289062
  tz: America/Edmonton

In [22]:
general_data,wind=openweather_adaptor.get_currentWeather(latitude=location1.latitude,longitude=location1.longitude)
general_data

OpenWeather_General(temp=11.27, feels_like=9.31, temp_min=11.27, temp_max=11.27, pressure=1019.0, humidity=33.0, sea_level=1019.0, grnd_level=938.0)

In [23]:
wind


OpenWeather_Wind(speed=2.21, deg=275.0, gust=4.05)

In [28]:
# get solcast data
irradiance_estimated=solcast_adaptor.get_estimated_actuals(latitude=location1.latitude,longitude=location1.longitude)
irradiance_estimated['period_end']=pd.to_datetime(irradiance_estimated['period_end'])
irradiance_estimated

,ghi,ebh,dni,dhi,cloud_opacity,period_end,period
0,408,301,660,107,10,2024-09-28 17:30:00+00:00,PT30M
1,401,353,859,48,0,2024-09-28 17:00:00+00:00,PT30M
2,341,296,828,45,0,2024-09-28 16:30:00+00:00,PT30M


In [38]:
from datetime import datetime,timezone
from zoneinfo import ZoneInfo
now=datetime.now(tz=ZoneInfo('UTC'))
irradiance_estimated["diff_from_now"]=irradiance_estimated['period_end'] - now
irradiance_estimated['diff_from_now']= irradiance_estimated['diff_from_now'].abs()
index_min=irradiance_estimated['diff_from_now'].idxmin()
# result_irradiance=irradiance_estimated.loc[index_min]
# result_irradiance.to_dict()
index_min

0

In [64]:
# in lib inverter files: https://github.com/NREL/SAM/tree/develop/deploy/libraries
# there are two ways of specifying data
from pvlib import pvsystem
modules = pvsystem.retrieve_sam('cecmod')
# customize parameters if module is not present in the database:
# https://pvlib-python.readthedocs.io/en/stable/user_guide/pvsystem.html#design-philosophy
# required weather options: 
# 1. Openweather API 
# 2. google solar API(geoTiff file)
# 3. Solcast

# customize parameters in source code: pvlib\tests\conftest.py


In [65]:
modules['AU_Optronics_PM060M00_275'].to_dict()

{'Technology': 'Mono-c-Si',
 'Bifacial': 0,
 'STC': 275.196,
 'PTC': 247.8,
 'A_c': 1.611,
 'Length': 1.639,
 'Width': 0.983,
 'N_s': 60,
 'I_sc_ref': 9.03,
 'V_oc_ref': 38.7,
 'I_mp_ref': 8.52,
 'V_mp_ref': 32.3,
 'alpha_sc': 0.005734,
 'beta_oc': -0.126781,
 'T_NOCT': 46.5,
 'a_ref': 1.659088,
 'I_L_ref': 9.032612,
 'I_o_ref': 6.641533e-10,
 'R_s': 0.170889,
 'R_sh_ref': 590.845032,
 'Adjust': 16.860441,
 'gamma_r': -0.4536,
 'BIPV': 'N',
 'Version': 'SAM 2018.11.11 r2',
 'Date': '1/3/2019'}

In [67]:
import pvlib
# df, _, _, metadata = pvlib.iotools.get_pvgis_tmy(location1.latitude, location1.longitude, map_variables=True)


In [68]:
# pvlib.iotools.get_pvgis_hourly(location1.latitude,location1.longitude)

In [69]:
from datetime import datetime,timedelta
from zoneinfo import ZoneInfo

# Get the current time in UTC
now_utc = datetime.now(ZoneInfo('UTC'))
print("Current UTC time:", now_utc)

# Convert UTC time to another timezone (e.g., America/Edmonton)
now_edmonton = now_utc.astimezone(ZoneInfo('America/Edmonton'))
print("Current Edmonton time:", now_edmonton)
now=datetime.now(ZoneInfo('America/Edmonton'))
offset=now.utcoffset()
offset.total_seconds()/3600

now.isoformat()
yesterday=now-timedelta(days=1)
df, meta = pvlib.iotools.solcast.get_solcast_historic(
    latitude=location1.latitude,
    longitude=location1.longitude,
    start=yesterday.isoformat(),
    duration="P1D",
    api_key=solcast_apikey,
    time_zone=(offset.total_seconds()/3600),
    period="PT60M"
)
df

Current UTC time: 2024-09-25 06:06:17.779739+00:00
Current Edmonton time: 2024-09-25 00:06:17.779739-06:00


,temp_air,dni,ghi
period_mid,,,
2024-09-24 00:30:00-06:00,11,0,0
2024-09-24 01:30:00-06:00,11,0,0
2024-09-24 02:30:00-06:00,11,0,0
2024-09-24 03:30:00-06:00,11,0,0
2024-09-24 04:30:00-06:00,11,0,0
2024-09-24 05:30:00-06:00,10,0,0
2024-09-24 06:30:00-06:00,11,0,0
2024-09-24 07:30:00-06:00,11,41,8
2024-09-24 08:30:00-06:00,13,504,105


In [70]:
yesterday.timestamp()

1727157977.779739

In [71]:
df.index

DatetimeIndex(['2024-09-24 00:30:00-06:00', '2024-09-24 01:30:00-06:00',
               '2024-09-24 02:30:00-06:00', '2024-09-24 03:30:00-06:00',
               '2024-09-24 04:30:00-06:00', '2024-09-24 05:30:00-06:00',
               '2024-09-24 06:30:00-06:00', '2024-09-24 07:30:00-06:00',
               '2024-09-24 08:30:00-06:00', '2024-09-24 09:30:00-06:00',
               '2024-09-24 10:30:00-06:00', '2024-09-24 11:30:00-06:00',
               '2024-09-24 12:30:00-06:00', '2024-09-24 13:30:00-06:00',
               '2024-09-24 14:30:00-06:00', '2024-09-24 15:30:00-06:00',
               '2024-09-24 16:30:00-06:00', '2024-09-24 17:30:00-06:00',
               '2024-09-24 18:30:00-06:00', '2024-09-24 19:30:00-06:00',
               '2024-09-24 20:30:00-06:00', '2024-09-24 21:30:00-06:00',
               '2024-09-24 22:30:00-06:00', '2024-09-24 23:30:00-06:00',
               '2024-09-25 00:30:00-06:00'],
              dtype='datetime64[ns, UTC-06:00]', name='period_mid', freq=None)